In [22]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.utils import shuffle
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,roc_curve, roc_auc_score,log_loss, classification_report,r2_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [25]:
os.environ['KAGGLE_USERNAME'] = 'vardhan.chebrolu'
os.environ['KAGGLE_KEY'] = '7ef605fc8dba5425d6965fbd4c8fbe1f'
! kaggle datasets download -d uciml/breast-cancer-wisconsin-data
! unzip breast-cancer-wisconsin-data.zip

Dataset URL: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data
License(s): CC-BY-NC-SA-4.0
breast-cancer-wisconsin-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


In [26]:
df=pd.read_csv('/content/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [27]:
df.isna()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
565,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
566,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
567,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [28]:
df['diagnosis'].value_counts()

,count
diagnosis,
B,357
M,212


In [29]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

X = df.drop(['diagnosis', 'id', 'Unnamed: 32'], axis=1)
y = df['diagnosis']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

numerical_features = X.columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

features_scaled = preprocessor.fit_transform(X)

print("Shape of original features:", X.shape)
print("Shape after preprocessing:", features_scaled.shape)

Shape of original features: (569, 30)
Shape after preprocessing: (569, 30)


In [30]:
ng = np.random.default_rng(7)
dataset = pd.read_csv('/content/data.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [31]:
feature_columns = ['radius_mean', 'area_mean', 'area_worst']
x = dataset[feature_columns]
y = dataset['diagnosis']
x, y = shuffle(x, y, random_state=42)
features = dataset[feature_columns]
labels = dataset['diagnosis']


In [32]:
features, labels = shuffle(features, labels, random_state=42)
train_features,test_features,train_labels,test_labels=train_test_split(features,labels,test_size=0.2,random_state=42)
shuffle_split_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

In [62]:
log_reg=LogisticRegression(max_iter=1000,random_state=42)
log_reg_pipeline = Pipeline([("feature_scaling", StandardScaler()), ("logistic_regression", LogisticRegression())])
log_reg_cv_results = cross_validate(log_reg_pipeline, train_features, train_labels, cv=shuffle_split_cv, scoring="accuracy")
log_reg_scores = pd.Series(log_reg_cv_results['test_score'], name="log_reg_accuracy")
log_reg_pipeline.fit(train_features, train_labels)
labels_pred = log_reg_pipeline.predict(test_features)
train_accuracy = accuracy_score(train_labels, log_reg_pipeline.predict(train_features))
test_accuracy = accuracy_score(test_labels, labels_pred)

find_accu= accuracy_score(test_labels,labels_pred)
find_prec= precision_score(test_labels,labels_pred, pos_label='M')
find_recall= recall_score(test_labels,labels_pred, pos_label='M')
find_f1= f1_score(test_labels,labels_pred, pos_label='M')
print("accuracy",find_accu)
print("precision",find_prec)
print("recall",find_recall)
print("f1",find_f1)
print("")

print("trained accuracy",train_accuracy)
print("test accuracy",test_accuracy)
print("")
if train_accuracy>test_accuracy+0.05:
  print("Overfitting")
elif train_accuracy<0.7 and test_accuracy<0.7:
  print("underfitting")
else:
  print("model is good")

accuracy 0.8859649122807017
precision 0.9459459459459459
recall 0.7608695652173914
f1 0.8433734939759037

trained accuracy 0.9252747252747253
test accuracy 0.8859649122807017

model is good


In [66]:
lin_reg = LinearRegression()
train_features, test_features, train_labels_encoded, test_labels_encoded = train_test_split(features, y_encoded, test_size=0.2, random_state=42)
lin_reg_pipeline = Pipeline([("linear_regression", LinearRegression())])
lin_reg_cv_results = cross_validate(lin_reg_pipeline, train_features, train_labels_encoded, cv=shuffle_split_cv, scoring="r2")
lin_reg_scores = pd.Series(lin_reg_cv_results['test_score'], name="lin_reg_r2")
lin_reg_pipeline.fit(train_features, train_labels_encoded)
labels_pred_encoded = lin_reg_pipeline.predict(test_features)
train_r2 = r2_score(train_labels_encoded, lin_reg_pipeline.predict(train_features))
test_r2 = r2_score(test_labels_encoded, labels_pred_encoded)
print("train R2:", train_r2)
print("test R2:", test_r2)
print("")
if train_r2 < 0.7 and test_r2 < 0.7:
  print("underfitting")
else:
  print("model fit is moderate to good")

train R2: 0.005445831891113251
test R2: -0.017398410911807938

underfitting


In [71]:
selected_features = ['radius_mean', 'area_mean', 'area_worst']
X = x[selected_features]
y = y
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Classes:", le.classes_)
print("Encoded target (first 10):", y_encoded[:10])


Classes: ['B' 'M']
Encoded target (first 10): [0 1 1 0 0 1 1 1 0 0]


In [72]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

model = LogisticRegression(max_iter=1000, solver='lbfgs')
model.fit(features_scaled, y)

probs = model.predict_proba(features_scaled)[:, 1]
loss = log_loss(y, probs)
final_weights = model.coef_[0]
final_bias = model.intercept_[0]

print("Forward Pass (first 5 probs):", probs[:5])
print("Backward Pass (final loss):", loss)
print("Final Weights shape:", final_weights.shape)
print("Final Bias:", final_bias)


Forward Pass (first 5 probs): [0.15844971 0.99985732 0.92706115 0.04055755 0.02554981]
Backward Pass (final loss): 0.20666709930172042
Final Weights shape: (3,)
Final Bias: 0.08434001672929425


In [73]:
loss=log_loss(test_labels,labels_pred_encoded)
print("loss",loss)
r2=r2_score(test_labels_encoded,labels_pred_encoded)
print("r2",r2)

loss 0.7308003135314127
r2 -0.017398410911807938
